In [175]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [176]:
tickers = {'TCS.NS','ICICIBANK.NS','ADANIGREEN.NS','HINDUNILVR.NS','INFY.NS','BHEL.NS'}
data = yf.download( tickers, start = '2024-02-07', end =None)['Adj Close']
#dat = yf.download( tickers, start = '2024-02-07', end =None)
#print(dat)
returns = data.pct_change().dropna()
#print(returns)
print(returns.std())
print(returns.mean())
Exp_return_general = returns.mean()
Exp_risk_general =returns.std()
C = returns.cov()
print(C)

C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[****************      33%%                      ]  2 of 6 completedC:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*****************

Ticker
ADANIGREEN.NS    0.031007
BHEL.NS          0.036309
HINDUNILVR.NS    0.013186
ICICIBANK.NS     0.015036
INFY.NS          0.012455
TCS.NS           0.011891
dtype: float64
Ticker
ADANIGREEN.NS   -0.000107
BHEL.NS          0.004182
HINDUNILVR.NS    0.000827
ICICIBANK.NS     0.002007
INFY.NS         -0.000058
TCS.NS          -0.000081
dtype: float64
Ticker         ADANIGREEN.NS   BHEL.NS  HINDUNILVR.NS  ICICIBANK.NS   INFY.NS  \
Ticker                                                                          
ADANIGREEN.NS       0.000961  0.000678      -0.000028      0.000255  0.000076   
BHEL.NS             0.000678  0.001318      -0.000126      0.000211  0.000025   
HINDUNILVR.NS      -0.000028 -0.000126       0.000174     -0.000018  0.000030   
ICICIBANK.NS        0.000255  0.000211      -0.000018      0.000226  0.000041   
INFY.NS             0.000076  0.000025       0.000030      0.000041  0.000155   
TCS.NS              0.000041 -0.000005       0.000031      0.000025  0.000102

In [177]:
w = np.ones(6)/6
C = np.array(C)

A = C @ w
MRC = A / np.sqrt(w.T @ C @ w)
print(MRC) 
def obv_function(w,C):
    value = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            value += ( w[i]*MRC[i] - w[j]*MRC[j])**2
    return value
def objective(w):
    return obv_function(w,C)



Constraints = (
    { 'type' : 'eq' , 'fun': lambda w: np.sum(w) -1},
    { 'type' : 'ineq', 'fun' : lambda w: w}
)      
bounds= [(0,1) for i in range(C.shape[1])]

answer = minimize(objective, w, method = 'SLSQP',constraints = Constraints, bounds = bounds, options={'disp': True} )
Optimal_weights = answer.x
print("optimal weigts :" ,Optimal_weights)
print("Objectve function value at optimal weights: ", answer.fun)
print("Success:" ,answer.success)
print("Messages:" ,answer.message)


print(returns.mean() @ w.T)

[0.02637421 0.02795477 0.00083795 0.00982922 0.00570955 0.00446153]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7760354799004706e-05
            Iterations: 5
            Function evaluations: 35
            Gradient evaluations: 5
optimal weigts : [0.0631286  0.03729666 0.2177189  0.22382861 0.22956628 0.22846095]
Objectve function value at optimal weights:  2.7760354799004706e-05
Success: True
Messages: Optimization terminated successfully
0.001128296909367997


In [187]:
w = np.ones(6)/6
C= np.array(C)
def obj_function(w,C):
    return np.sqrt(w.T @ C @ w)
def objective(w):
    return obj_function(w,C)

constraints = (
    { 'type' : 'ineq', 'fun' : lambda w : np.sum(np.log(w)) + 2},
    { 'type' : 'ineq' , 'fun' : lambda w  : w}
)
bounds = [(0,1) for i in range(C.shape[1])]

results = minimize(objective, w,  method = 'SLSQP', constraints = constraints, bounds = bounds, options = { 'disp' : True})
Op = results.x
print("optimal weights are" ,Op)
function_value = results.fun
print("optimal value of function at optimal weights" , function_value)
print(results.message)
print(results.success)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.043831065340953455
            Iterations: 14
            Function evaluations: 99
            Gradient evaluations: 14
optimal weights are [0.37971743 0.41629834 1.         0.86705349 0.98741516 1.        ]
optimal value of function at optimal weights 0.043831065340953455
Optimization terminated successfully
True


In [190]:
total =0
for i in range(0, len(Op)):
    total = total + Op[i]
print(total)
print(np.sum(Op))
New_weights = Op/total
print(New_weights)

4.650484428026193
4.650484428026193
[0.08165116 0.0895172  0.21503136 0.18644369 0.21232523 0.21503136]


In [193]:
print(returns.mean() @ New_weights.T)

Exp_returns_method2 = returns.mean() @ New_weights.T
Exp_risk_method2 = New_weights.T @ C @ New_weights

0.0008878823046197032


In [199]:
print(New_weights.T @ C @ New_weights)
print(Exp_returns_method2)

8.883163674442668e-05
0.0008878823046197032


In [194]:
tickers = {'TCS.NS','ICICIBANK.NS','ADANIGREEN.NS','HINDUNILVR.NS','INFY.NS','BHEL.NS'}
Market_cap = [14.52e12, 8.68e12, 2.78e12, 5.98e12,6.84e12,1.10e12]
total_market_cap = np.sum(Market_cap) 
Weighted_market_cap = Market_cap/total_market_cap
print(Weighted_market_cap)
W = Weighted_market_cap

[0.36390977 0.21754386 0.06967419 0.14987469 0.17142857 0.02756892]


In [195]:
Exp_return_market = returns.mean().T @ W
print(Exp_return_market)

0.001216953828235251


In [196]:
Exp_risk_market = W.T @ C @ W
print(Exp_risk_market)

0.0003597469280150585


In [197]:
Risk_free_rate = 0.0007
sharpe_market_cap = (Exp_return_market - Risk_free_rate) / Exp_risk_market
print(sharpe_market_cap)

1.436993030316056


In [198]:
Sharpe_method2 =(Exp_returns_method2 - Risk_free_rate) / Exp_risk_method2
print(Sharpe_method2)

2.115038194784708
